In [3]:
!pip install gspread

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 2.3 MB/s eta 0:00:00


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

# Setup Google Sheets credentials
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("your-creds.json.json", scope)
client = gspread.authorize(creds)

sheet = client.open("BitcoinRealtimeData").sheet1  # Create this sheet manually first

# Function to push data
# Function to push data to the sheet
def push_to_sheet(timestamp, avg_price, prediction=None):
    if prediction is None:
        sheet.append_row([timestamp, avg_price])  # Only append timestamp and avg_price if no prediction
    else:
        sheet.append_row([timestamp, avg_price, prediction])  # Append all three if prediction exists

import threading
from confluent_kafka import DeserializingConsumer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroDeserializer
from confluent_kafka.serialization import StringDeserializer
from datetime import datetime
import numpy as np
from tensorflow.keras.models import load_model

# Load model
model = load_model("bitcoin_lstm_model_with_buffer_Mark-2.keras")

# Define Kafka configuration
kafka_config = {
    'bootstrap.servers': 'Cluster-server',
    'sasl.mechanisms': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': 'Cluster-name',
    'sasl.password': 'Cluster-password',
    'group.id': '1'
}

# Create a Schema Registry client
schema_registry_client = SchemaRegistryClient({
  'url': 'Schema-registry-endpoint',
  'basic.auth.user.info': '{}:{}'.format('Schema-registry-name', 'Schema-registry-password')
})

# Fetch the latest Avro schema for the value
subject_name = 'bitcoin_stream-value'
schema_str = schema_registry_client.get_latest_version(subject_name).schema.schema_str

# Create Avro Deserializer for value
key_deserializer = StringDeserializer('utf_8')
avro_deserializer = AvroDeserializer(schema_registry_client, schema_str)

# Define the DeserializingConsumer
consumer = DeserializingConsumer({
    'bootstrap.servers': kafka_config['bootstrap.servers'],
    'security.protocol': kafka_config['security.protocol'],
    'sasl.mechanisms': kafka_config['sasl.mechanisms'],
    'sasl.username': kafka_config['sasl.username'],
    'sasl.password': kafka_config['sasl.password'],
    'key.deserializer': key_deserializer,
    'value.deserializer': avro_deserializer,
    'group.id' : kafka_config['group.id'],
    'auto.offset.reset': 'latest',
    'enable.auto.commit': True,
    'auto.commit.interval.ms': 5000  # Commit every 5000 ms
})

# Subscribe to topic
consumer.subscribe(['bitcoin_stream'])

minute_prices = []  # Prices within the current minute
minute_buffer = []  # Stores 1-min avg prices for the last 60 minutes
current_minute = None
WINDOW_SIZE = 60  # 60 minutes
BUFFER_PERCENT = 0.05

print("Listening and aggregating...")

try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            continue
        if msg.error():
            print('Consumer error: {}'.format(msg.error()))
            continue

        # Deserialize message
        msg_data = msg.value()
        timestamp_str = msg_data.get("timestamp")
        price = float(msg_data.get("price"))  # Assuming price is in the message

        # Parse timestamp and handle minute-based aggregation
        dt = datetime.fromisoformat(timestamp_str)
        minute_key = (dt.year, dt.month, dt.day, dt.hour, dt.minute)

        if current_minute is None:
            current_minute = minute_key

        if minute_key == current_minute:
            minute_prices.append(price)
        else:
            # Process previous minute's data
            if minute_prices:
                avg_price = sum(minute_prices) / len(minute_prices)
                minute_buffer.append(avg_price)

                print(f"[{current_minute}] 1-min avg: {avg_price:.2f}")

                # Prepare to send prediction if available
                prediction_to_send = None

                # If enough data for prediction
                if len(minute_buffer) == WINDOW_SIZE:
                    window = np.array(minute_buffer[-WINDOW_SIZE:])
                #if len(minute_buffer) >= WINDOW_SIZE + 2:
                 #   window = np.array(minute_buffer[-(WINDOW_SIZE + 2):-2])  # Last 60 mins
                    row_min = np.min(window)
                    row_max = np.max(window)
                    range_ = row_max - row_min

                    if range_ != 0:
                        buffer = BUFFER_PERCENT * range_
                        row_min -= buffer
                        row_max += buffer

                        window_scaled = (window - row_min) / (row_max - row_min)
                        window_scaled = window_scaled.reshape((1, WINDOW_SIZE, 1))

                        # Predict using model
                        scaled_pred = model.predict(window_scaled, verbose=0)[0][0]
                        actual_pred = scaled_pred * (row_max - row_min) + row_min
                        prediction_to_send = actual_pred

                        # Print prediction
                        print(f"\n📈 Predicted Bitcoin Price (2 mins ahead): ₹{actual_pred:.2f}\n")
                        minute_buffer = minute_buffer[2:]

                # Push to sheet
                push_to_sheet(str(current_minute), avg_price, prediction_to_send)

            # Reset for new minute
            current_minute = minute_key
            minute_prices = [price]

except KeyboardInterrupt:
    pass
finally:
    consumer.close()


2025-04-28 13:01:20.121508: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-28 13:01:22.537564: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-28 13:01:23.086702: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745845283.704036    3464 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745845283.935884    3464 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745845286.157765    3464 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

Listening and aggregating...


%6|1745845312.473|GETSUBSCRIPTIONS|rdkafka#consumer-1| [thrd:main]: Telemetry client instance id changed from AAAAAAAAAAAAAAAAAAAAAA to pJ+b3LE2Tq6uvBzCQnTXwA


[(2025, 4, 28, 18, 32)] 1-min avg: 95320.64
[(2025, 4, 28, 18, 33)] 1-min avg: 95320.64
[(2025, 4, 28, 18, 34)] 1-min avg: 95320.65
[(2025, 4, 28, 18, 35)] 1-min avg: 95320.69
[(2025, 4, 28, 18, 36)] 1-min avg: 95322.68
[(2025, 4, 28, 18, 37)] 1-min avg: 95330.73
[(2025, 4, 28, 18, 38)] 1-min avg: 95340.07
[(2025, 4, 28, 18, 39)] 1-min avg: 95345.76
[(2025, 4, 28, 18, 40)] 1-min avg: 95355.03
[(2025, 4, 28, 18, 41)] 1-min avg: 95366.90
[(2025, 4, 28, 18, 42)] 1-min avg: 95379.25
[(2025, 4, 28, 18, 43)] 1-min avg: 95395.08
[(2025, 4, 28, 18, 44)] 1-min avg: 95397.64
[(2025, 4, 28, 18, 45)] 1-min avg: 95388.32
[(2025, 4, 28, 18, 46)] 1-min avg: 95378.18
[(2025, 4, 28, 18, 47)] 1-min avg: 95369.63
[(2025, 4, 28, 18, 48)] 1-min avg: 95365.80
[(2025, 4, 28, 18, 49)] 1-min avg: 95360.58
[(2025, 4, 28, 18, 50)] 1-min avg: 95352.50
[(2025, 4, 28, 18, 51)] 1-min avg: 95347.49
[(2025, 4, 28, 18, 52)] 1-min avg: 95341.76
[(2025, 4, 28, 18, 53)] 1-min avg: 95334.31
[(2025, 4, 28, 18, 54)] 1-min av

[(2025, 4, 28, 20, 50)] 1-min avg: 94373.09
[(2025, 4, 28, 20, 51)] 1-min avg: 94373.93

📈 Predicted Bitcoin Price (2 mins ahead): ₹94464.38

[(2025, 4, 28, 20, 52)] 1-min avg: 94374.02
[(2025, 4, 28, 20, 53)] 1-min avg: 94379.95

📈 Predicted Bitcoin Price (2 mins ahead): ₹94467.72

